패키지 로드

In [1]:
import numpy as np
import pandas as pd
import time
import re
from datetime import datetime, timedelta

데이터 로드

In [ ]:
df = pd.read_csv("C:/study_data/project/insta/중간자료/totaldata.csv", index_col=0)

중복 댓글 제거

In [ ]:
# 크롤링 과정에서 중복으로 가져온 댓글 제거 (댓글 89만개 -> 65만개)
df = df.drop_duplicates(subset='reply', keep='first')

In [ ]:
# 꼬인 인덱스 수정
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)

광고글 라벨링

In [ ]:
# 광고글인지 판단하는 함수
def chk_adv(main_txt):
    temp = 0
    promo_word = ['광고', '협찬', '제품제공', 'AD', 'GIFTED', '제공', '제품협찬', '유료광고', '상업광고', '금전적']
    for w in promo_word:
        if w in main_txt:
            temp = 1
        if temp ==1:
            break
    return temp
df['promotion'] = df['content_text'].apply(lambda x : chk_adv(str(x)))

해시태그 추출

In [ ]:
# 해시태그 단어 컬럼 추가 : #word
hashtag_regex = "#([0-9a-zA-Z가-힣]*)"
p = re.compile(hashtag_regex)
df["#word"] = df["content_text"].apply(lambda x : p.findall(str(x)))

### 게시글 및 댓글 전처리

텍스트 전처리 함수

In [ ]:
# 전처리 함수(글자, 영어, 숫자만 남기기)
def text_preprocessor(s):
    import re
    
    ## (1) [], (), {}, <> 괄호 제거
    s = str(s).replace('(', ' ')
    s = str(s).replace(')', ' ')
    s = str(s).replace('[', ' ')
    s = str(s).replace(']', ' ')
    s = str(s).replace('<', ' ')
    s = str(s).replace('>', ' ')
    s = str(s).replace('{', ' ')
    s = str(s).replace('}', ' ')

    hashtag_regex = "#([0-9a-zA-Z가-힣_.]*)"
    s = re.sub(pattern=hashtag_regex, repl='', string=s)
    
    call_regex = "@([0-9a-zA-Z가-힣_.]*)"
    s = re.sub(pattern=call_regex, repl='', string=s)


    ## (2) '...외', '...총' 제거하기
    s = s.replace('...외', ' ')
    s = s.replace('...총', ' ')
    
    ## (3) 특수문자 제거
    pattern = r'[^a-zA-Z가-힣]'
    s = re.sub(pattern=pattern, repl=' ', string=s)
    
    ## (4) 단위 제거: cm, km, etc.
    units = ['mm', 'cm', 'km', 'ml', 'kg', 'g']
    for unit in units:
        s = s.lower() # 대문자를 소문자로 변환
        s = s.replace(unit, '')
        
    # (5) 공백 지워서 한줄로 합치기
    s = s.replace(' ', '')

    return s

In [ ]:
# 게시글 전처리함수 적용
df['pre_content'] = df['content_text'].apply(lambda s: text_preprocessor(s))

In [ ]:
# 게시글 대문자화(브랜드명 추출할 때 사용)
df['pre_content'] = df.loc[:, 'pre_content'].apply(lambda x: x.upper())
# 해시태그 대문자화
df['#word'] = df.loc[:,'#word'].apply(lambda x : str(x).upper())

In [ ]:
# 댓글 전처리함수 적용
df['pre_reply'] = df['reply'].apply(lambda s: text_preprocessor(s))
# 전처리된 'reply' 대문자화
df['pre_reply'] = df.loc[:, 'pre_reply'].apply(lambda x: x.upper())

### 컬럼 추가

게시글 글자수 컬럼 추가

In [ ]:
# 게시물 글자수 컬럼 추가 : content_text_len
def len_text(t):
    try:
        len_txt = len(t)
    except:
        len_txt = 0
    return len_txt
df['content_text_len'] = df['content_text'].apply(len_text)

인플루언서별 정보 컬럼 추가
- 광고글/비광고글 수
- 전체 게시글 좋아요/댓글 평균
- 광고글/비광고글 좋아요/댓글 평균

In [ ]:
# 광고글 수 : promo_content_num
df['promo_content_num'] = 0
for id in df['name'].unique():
    df.loc[df['name'] == id, 'promo_content_num'] = len(df[(df['name']==id) & (df['promotion']==1)]['content_text'].unique())
# 비광고글 수 : nopromo_content_num
df['nopromo_content_num'] = df['content_num'] - df['promo_content_num']

In [ ]:
# 인플루언서별 전체 게시글 좋아요/댓글 수 평균 : all_con_like_mean, all_con_rp_mean
df_name_mean = df.groupby(['name']).agg({'content_like':'mean', 'reply_tot':'mean'})
df_name_mean.rename(columns={'content_like':'all_con_like_mean', 'reply_tot':'all_con_rp_mean'}, inplace=True)
df = pd.merge(df, df_name_mean, how='inner', on='name')

In [ ]:
# 광고/비광고글 좋아요/댓글 평균 추가 : promo_con_like_mean, promo_con_rp_mean, nopromo_con_like_mean, nopromo_con_rp_mean
promo_mean = df[df['promotion']==1].groupby(['name']).agg({'content_like':'mean', 'reply_tot':'mean'})
promo_mean.rename(columns={'content_like':'promo_con_like_mean', 'reply_tot':'promo_con_rp_mean'}, inplace=True)
nopromo_mean = df[df['promotion']==0].groupby(['name']).agg({'content_like':'mean', 'reply_tot':'mean'})
nopromo_mean.rename(columns={'content_like':'nopromo_con_like_mean', 'reply_tot':'nopromo_con_rp_mean'}, inplace = True)
df = pd.merge(df, promo_mean, how='inner', on='name')
df = pd.merge(df, nopromo_mean, how='inner', on='name')

댓글 시간 관련 컬럼 추가 (분 단위로 변환)
- 댓글 반응시간
- 댓글 최소 반응시간
- 댓글 반응 기간

In [75]:
# 댓글 시간/게시글 시간 : 시간 형식으로 변환
df['reply_time'] = df['reply_time'].apply(lambda x:datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df['content_time'] = df['content_time'].apply(lambda x:datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [ ]:
# 댓글 반응시간 컬럼 추가 : react_time
df['react_time'] = df['reply_time'] - df['content_time']

In [ ]:
# 게시글별 댓글 최소 반응시간 : react_time_min
df['react_time_min'] = 0
cnt_num = 0
for w in df['content_time'].unique():
    df.loc[df['content_time'] == w, 'react_time_min'] = min(df[df['content_time'] == w]['react_time'])
    cnt_num += 1
    print(cnt_num, '/', len(df['content_time'].unique()), 'fin---')

In [ ]:
# 게시글별 댓글 반응기간 : react_term
df['react_term'] = 0
cnt_num = 0
for w in df['content_time'].unique():
    df.loc[df['content_time'] == w, 'react_term'] = max(df[df['content_time'] == w]['react_time']) - min(df[df['content_time'] == w]['react_time'])
    cnt_num += 1
    print(cnt_num, '/', len(df['content_time'].unique()), 'fin---')

In [ ]:
# 반응시간 / 반응기간 / 최소 반응시간 분단위로 변경
df['react_time'] = df['react_time'].apply(lambda x:x.seconds/60)
df['react_term'] = df['react_term'].apply(lambda x:x.seconds/60)
df['react_time_min'] = df['react_time_min'].apply(lambda x:x.seconds/60)

게시글 업로드 관련 컬럼 추가
- 게시글 업로드 요일
- 게시글 업로드 시각

In [ ]:
# 게시날짜 요일로 변환해 추가 : con_weekday
def week(date_str):
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    return days[date_str.weekday()]
df['con_weekday'] = df['content_time'].apply(lambda x:week(x))

In [ ]:
# 게시시각 컬럼 추가 : con_hour
df['con_hour'] = df['content_time'].apply(lambda x:x.hour)

언급/해시태그 수 컬럼 추가

In [ ]:
# 언급(@) 수 컬럼 추가 : mention_num
df['mention_num'] = df['reply'].apply(lambda x:0 if type(x)!=str else len(re.findall(r'@', x)))

In [ ]:
# 해시태그(#) 수 컬럼 추가 : hashtag_num
df['hashtag_num'] = df['content_text'].apply(lambda x:0 if type(x)!=str else len(re.findall(r'#', x)))

날짜 가중치 컬럼 추가

In [73]:
# 날짜 가중치 추가
temp = df['name'].unique()
influ = temp.tolist()
df['date_weight'] = 0.0

In [ ]:
# 날짜 가중치 컬럼 추가
for name in influ:
    temp_df = df[df['name'] == name]
    max_date = temp_df['content_time'].max()
    min_date = temp_df['content_time'].min()
    delta1 = max_date - min_date
    under = delta1.total_seconds() / 60
    
    for i in range(0, len(temp_df) - 1):
        std_date = temp_df.iloc[i, 6]
        delta2 = std_date - min_date
        up = delta2.total_seconds() / 60
        res = up/under
        print(i, "번째: ", res)

        df.loc[temp_df.index[i], 'date_weight'] = res

데이터프레임 저장

In [ ]:
df.to_csv("C:/study_data/project/insta/중간자료/insta_reply_data.csv")